# 아이오와 집값 데이터 분석하기 

In [6]:
import warnings
warnings.filterwarnings(action='ignore')

# 밑에 실행하면서 뜨는 빨간 경고 팝업 뜨지 않게 하기 

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
data = pd.read_csv("../07deep_learning/data/house_test.csv")
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [9]:
from tensorflow.keras.models import load_model

In [10]:
model = load_model("./model/iowa_house757__923928384.0000.keras")

2024-09-11 15:08:32.659805: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-11 15:08:32.661145: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 0 (AMD Radeon(TM) Graphics)
2024-09-11 15:08:33.053292: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-11 15:08:33.053353: W tensorflow/core/common_runtime/pluggable_device/pluggable_device_bfc_allocator.cc:28] Overriding allow_growth setting because force_memory_growth was requested by the device.
2024-09-11 15:08:33.053389: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

In [12]:
data2 = data.copy()
data2 = pd.get_dummies(data2)

In [13]:
data2.info()

# 다 숫자로 바뀌었으니 이제 스케일링 가능하다

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 270 entries, Id to SaleCondition_Partial
dtypes: bool(233), float64(11), int64(26)
memory usage: 753.8 KB


In [15]:
data2_selected = data2[['OverallQual', 'GrLivArea', 'GarageCars', 'YearBuilt', 'FullBath',
       'TotalBsmtSF', '1stFlrSF', 'ExterQual_Gd', 'YearRemodAdd', 'Fireplaces',
       'KitchenQual_TA', 'GarageFinish_Unf', 'ExterQual_TA']]


04_아이오아 집값데이터에서 <house train data> 를 가지고 분석했다.
<br><br>
"이전에 적용했던 스케일링을 TEST 데이터에도 적용시킨다"는 말은 훈련 데이터(train data)에 사용했던 스케일링 기법을 그대로 테스트 데이터(test data)에도 적용한다는 의미다. 이 과정은 머신러닝 모델의 정확성을 보장하기 위해 중요한 과정이다.


스케일링은 각 피처(변수)를 일정한 범위(예: 0과 1 사이)로 변환하는 과정이다. 일반적으로 MinMaxScaler, StandardScaler 등의 방법이 사용된다. 스케일링은 훈련 데이터에 대해서 먼저 적용된다. 즉, 훈련 데이터의 최소값과 최대값 또는 평균과 표준편차를 기준으로 데이터가 변환된다.


* 왜 TEST 데이터에도 적용해야 하는가? <br>
훈련 데이터와 테스트 데이터가 서로 다른 범위로 스케일링되면, 모델이 훈련과 테스트에서 다른 스케일의 데이터를 보고 혼란스러워 할 수 있다. 테스트 데이터를 스케일링할 때는 훈련 데이터에서 계산한 스케일 기준(예: 평균, 표준편차)을 그대로 사용해야 한다. 테스트 데이터를 기준으로 새롭게 스케일링을 하면 안 된다. 


이렇게 해야 테스트 데이터가 훈련 데이터와 동일한 스케일로 변환되므로 모델이 일관성 있게 동작할 수 있다. 

## joblib 으로 저장한 scaler, model 불러오기

* 04_아이오아 집값데이터에서 를 가지고 분석했다.
* 04번 파일에서 joblib으로 robust scaler 모델 저장 후 
<br><br>

* 05번 파일에서 joblib으로 모델 불러와서 사용한다. 

In [16]:
import joblib

In [17]:
robust_scaler = joblib.load("./model/iowa_robustscaler.joblib")

In [22]:
data2_scaled = robust_scaler.transform(data2_selected)
data2_scaled = pd.DataFrame(data2_scaled, columns=data2_selected.columns)
data2_scaled 

,OverallQual,GrLivArea,GarageCars,YearBuilt,FullBath,TotalBsmtSF,1stFlrSF,ExterQual_Gd,YearRemodAdd,Fireplaces,KitchenQual_TA,GarageFinish_Unf,ExterQual_TA
0,-0.5,-0.877559,-1.0,-0.260870,-1.0,-0.217910,-0.375061,0.0,-0.891892,-1.0,0.0,1.0,0.0
1,0.0,-0.208575,-1.0,-0.326087,-1.0,0.671642,0.475209,0.0,-0.972973,-1.0,-1.0,1.0,0.0
2,-0.5,0.254925,0.0,0.521739,0.0,-0.126368,-0.312224,0.0,0.108108,0.0,0.0,0.0,0.0
3,0.0,0.216300,0.0,0.543478,0.0,-0.130348,-0.316151,0.0,0.108108,0.0,-1.0,0.0,0.0
4,1.0,-0.284280,0.0,0.413043,0.0,0.574129,0.378989,1.0,-0.054054,-1.0,-1.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,-1.0,-0.574739,-2.0,-0.065217,-1.0,-0.886567,-1.062347,0.0,-0.648649,-1.0,0.0,0.0,0.0
1455,-1.0,-0.574739,-1.0,-0.065217,-1.0,-0.886567,-1.062347,0.0,-0.648649,-1.0,0.0,1.0,0.0
1456,-0.5,-0.370800,0.0,-0.282609,-1.0,0.462687,0.269023,0.0,0.054054,0.0,0.0,1.0,0.0
1457,-0.5,-0.763229,-2.0,0.413043,-1.0,-0.158209,-0.229750,0.0,-0.054054,-1.0,0.0,0.0,0.0


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data2_scaled, y, test_size=0.3, random_state=10)

In [ ]:
#X = data.drop("SalePrice", axis=1)
#y = data['SalePrice']

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_model = XGBClassifier()